In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion, Pipeline

from scipy.sparse import csr_matrix
from scipy.sparse import hstack

In [2]:
train = pd.read_csv('data/evo_train.csv.gz', compression='gzip', index_col='id')
categories = pd.read_csv('data/categories_parsed.csv.gz', compression='gzip', index_col='GROUP_ID')
data = train.join(categories, on='GROUP_ID')
data.head()

,NAME,GROUP_ID,category,sub_category
id,,,,
0,"Пиво ""Жигулевское"" 0,5 л. св.",35,Алкоголь,"Пиво, слабоалкогольные напитки"
1,СОУС ТОМАТНЫЙ БУЗДЯК 670Г ТАТАРСКИЙ /8,6,Прод,Продукты питания
2,Сигареты Esse SS Exchange,7,Непрод,Табачные изделия
3,Петрушка,6,Прод,Продукты питания
4,пиво ягерь,35,Алкоголь,"Пиво, слабоалкогольные напитки"


In [3]:
X = data['NAME']
y = data['category']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

test = pd.DataFrame(index=y_test.index)
test['NAME'] = X_test
test['category'] = y_test

In [7]:
pipeline = Pipeline([
        ('count_vectorizer', CountVectorizer()),
        ('clf_category', MultinomialNB())
    ])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print(classification_report(y_test, pred))

                           precision    recall  f1-score   support

                 Алкоголь       0.98      0.96      0.97      3583
                   Непрод       0.96      0.99      0.98     11214
Позиция по свободной цене       0.00      0.00      0.00         4
                     Прод       0.95      0.84      0.89      1774
                      н/д       0.99      0.84      0.91       183

              avg / total       0.97      0.97      0.97     16758



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
pred_df = pd.DataFrame(pred, index=X_test.index,columns=['predicted'])
pred_df['real'] = y_test
pred_df['NAME'] = X_test
pred_df[pred_df['real'] != pred_df['predicted']]

,predicted,real,NAME
id,,,
6561,Непрод,Алкоголь,"БИТОКСИБАЦИЛЛИН, П/Август/20гр"
48333,Непрод,Алкоголь,набор подарочный для вина
50666,Непрод,Прод,Бутерброд №44
43089,Непрод,Прод,Малена / Акконд
21696,Непрод,Алкоголь,Лавила Нанни сух.150
49520,Прод,Непрод,Кофе Hausbrandt Nero Espresso
54820,Непрод,Алкоголь,451°по Фаренгейту/Р.Брэдбери
48764,Непрод,Прод,шарлотта земляничка
30643,Непрод,Алкоголь,Вал карданный 3307


In [10]:
from nltk.stem.snowball import RussianStemmer

stemmer = RussianStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(analyzer=stemmed_words)

In [11]:
pipeline = Pipeline([
        ('count_vectorizer', stem_vectorizer),
        ('clf_category', MultinomialNB())
    ])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print(classification_report(y_test, pred))

                           precision    recall  f1-score   support

                 Алкоголь       0.97      0.96      0.97      3583
                   Непрод       0.97      0.99      0.98     11214
Позиция по свободной цене       0.00      0.00      0.00         4
                     Прод       0.94      0.85      0.89      1774
                      н/д       0.99      0.84      0.91       183

              avg / total       0.96      0.96      0.96     16758



/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
stem_vectorizer.vocabulary_

{'суста': 27607,
 'тягов': 28494,
 'точечн': 28262,
 'beverl': 6552,
 '8v': 5566,
 'ярмарочн': 30940,
 'джилл': 17067,
 'микромед': 21556,
 'tuborg': 12454,
 '8w': 5567,
 'канальн': 18646,
 'boilermaker': 6685,
 'lass': 9507,
 'хемингуэ': 29489,
 '0208575670': 163,
 'кустарников': 20153,
 'толстовк': 28156,
 'пикур': 23837,
 '5х7х22': 4507,
 'любин': 20786,
 'головкой8': 16435,
 '15х1500': 1518,
 'rtf': 11386,
 'миринд': 21654,
 'велсофт': 15345,
 'vp': 12746,
 'педигр': 23589,
 'яч': 30966,
 'липучк': 20567,
 'herbamedicus': 8825,
 '371': 3340,
 'кисел': 19094,
 'arabic': 6209,
 '94022368': 5758,
 'беловод': 14412,
 'basar': 6447,
 'jewelr': 9161,
 'потер': 24536,
 'с126тер': 25937,
 'b33': 6358,
 'полян': 24415,
 'bi': 6560,
 'альг': 13532,
 'флорид': 29152,
 'застольн': 17955,
 'барнаульск': 14245,
 'bright': 6778,
 '11336': 878,
 'чег': 29899,
 't5mv60elc': 12194,
 'строббс': 27462,
 'pre': 10986,
 'hond': 8888,
 'кэт': 20188,
 'кат': 18865,
 '515кг': 4175,
 'sf': 11649,
 'человече

In [12]:
pred_df = pd.DataFrame(pred, index=X_test.index,columns=['predicted'])
pred_df['real'] = y_test
pred_df['NAME'] = X_test
pred_df[pred_df['real'] != pred_df['predicted']]

,predicted,real,NAME
id,,,
6561,Непрод,Алкоголь,"БИТОКСИБАЦИЛЛИН, П/Август/20гр"
48333,Непрод,Алкоголь,набор подарочный для вина
50666,Непрод,Прод,Бутерброд №44
43089,Непрод,Прод,Малена / Акконд
21696,Непрод,Алкоголь,Лавила Нанни сух.150
49520,Прод,Непрод,Кофе Hausbrandt Nero Espresso
54820,Непрод,Алкоголь,451°по Фаренгейту/Р.Брэдбери
51240,Непрод,Прод,Слимо с арахисом
48764,Непрод,Прод,шарлотта земляничка
